In [33]:
import pandas as pd
import pydeck as pdk
from ODPworkspace_demotools import mapOQS, pydeck_plot
import matplotlib.pyplot as plt
from shapely.geometry import shape, MultiPolygon, Polygon
from shapely import wkt
from time import time
import geopandas as gpd
import os
from rasterio.io import MemoryFile
from IPython.display import display
from ipywidgets import widgets
from IPython.display import clear_output
import base64
import io
import warnings

In [24]:
from odp.client import OdpClient
from odp.dto import Metadata
from odp.dto.catalog import DatasetDto, DatasetSpec
from odp.dto.common.contact_info import ContactInfo

In [25]:
client = OdpClient()

In [26]:
dataset = client.catalog.get(("8131a076-af24-483b-b9a1-20a37433ef46")) ## UUID of ProtectedSeas dataset from the catalog
dataset.metadata.display_name

/srv/conda/envs/notebook/lib/python3.11/site-packages/pydantic/main.py:1159: PydanticDeprecatedSince20: The `parse_obj` method is deprecated; use `model_validate` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.9/migration/
  warnings.warn(


'ProtectedSeas MPA Dataset'

### List the Table Schema of ProtectedSeas Database

In [27]:
PSmetadata = client.tabular.get_schema(dataset)

/srv/conda/envs/notebook/lib/python3.11/site-packages/pydantic/main.py:1179: PydanticDeprecatedSince20: The `parse_raw` method is deprecated; if your data is JSON use `model_validate_json`, otherwise load the data then use `model_validate` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.9/migration/
  warnings.warn(
/srv/conda/envs/notebook/lib/python3.11/site-packages/pydantic/main.py:1187: PydanticDeprecatedSince20: `load_str_bytes` is deprecated. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.9/migration/
  obj = parse.load_str_bytes(


In [28]:
table_schema = PSmetadata.table_schema

schema_data = []
for column, details in table_schema.items():
    schema_data.append({
        "Column Name": column,
        "Type": details.get('type', 'Unknown'),
        "Nullable": details.get('nullable', False),
        "Metadata": details.get('metadata', {})
    })

df = pd.DataFrame(schema_data)
df

,Column Name,Type,Nullable,Metadata
0,ogc_fid,long,True,{}
1,diving_prohibited,long,True,{}
2,regulation_url,string,True,{}
3,s57_cat,double,True,{}
4,site_id,string,True,{}
5,gov_level,string,True,{}
6,longlining,long,True,{}
7,commercial_restrictions,long,True,{}
8,construction_prohibited,long,True,{}
9,recreational_restrictions,long,True,{}


### Filter by Country

In [30]:

# Suppress all warnings
warnings.filterwarnings("ignore")

# Prompt the user to input a country name
country_name = input("Enter the country name: ")

# Create the filter query with the user input
filter_query = {
  "#EQUALS": [
    "$country",
    country_name
  ]
}

# Query the data using the filter
PSdata = client.tabular.select_as_dataframe(dataset, filter_query=filter_query)

# Display the data
PSdata

Enter the country name:  Ireland


,regulation_url,boundary_source,inshore_only,definitions,anchoring_prohibited,construction_prohibited,hook_n_line,dredging_prohibited,trolling,tribal,...,site_name,coastline_match,latest_updates,industrial_or_mineral_exploration_prohibited,season,geometry_reduced,state,longlining,entry_prohibited,regulation_name
0,S.I. No. 86/2014- Maritime Jurisdiction (Bound...,Flanders Marine Institute (2019). Maritime Bou...,0.0,None,3,2,3,3,2,1,...,Ireland Territorial Sea,1.0,For updates please see the Protected Sites in ...,2,Year-round,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",None,3,3,S.I. No. 86/2014- Maritime Jurisdiction (Bound...
1,S.I. No. 86/2014- Maritime Jurisdiction (Bound...,Flanders Marine Institute (2019). Maritime Bou...,0.0,None,3,2,3,3,2,1,...,Ireland EEZ,1.0,For updates please see the Protected Sites in ...,2,Year-round,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",None,3,3,S.I. No. 86/2014- Maritime Jurisdiction (Bound...
2,Regulations Text|http://www.irishstatutebook.i...,National Park and Wildlife Service,0.0,None,3,2,3,2,3,1,...,River Shannon and River Fergus Estuaries SPA,1.0,For updates please see the Protected Sites in ...,2,Year-round,"{'type': 'MultiPolygon', 'coordinates': [[[[-8...",None,3,3,S.I. No. 329 of 2019- EUROPEAN UNION CONSERVAT...
3,http://www.irishstatutebook.ie/eli/2019/si/95/...,National Park and Wildlife Service,0.0,None,3,2,3,2,3,1,...,West Connacht Coast SAC,1.0,For updates please see the Protected Sites in ...,2,Year-round,"{'type': 'MultiPolygon', 'coordinates': [[[[-9...",None,3,3,S.I. No. 95 of 2019- EUROPEAN UNION HABITATS (...
4,Regulations Text|http://www.irishstatutebook.i...,National Park and Wildlife Service,0.0,None,3,2,3,3,3,1,...,Illancrone and Inishkeeragh SPA,0.0,For updates please see the Protected Sites in ...,3,Year-round,"{'type': 'MultiPolygon', 'coordinates': [[[[-8...",None,3,3,S.I. No. 66/2010 - European Communities (Conse...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186,Regulations Text|http://www.irishstatutebook.i...,National Park and Wildlife Service,0.0,None,3,2,3,2,3,1,...,Kerry Head Shoal SAC,0.0,For updates please see the Protected Sites in ...,2,Year-round,"{'type': 'MultiPolygon', 'coordinates': [[[[-1...",None,3,3,S.I. No. 102 of 2016- EUROPEAN UNION HABITATS ...
187,https://eur-lex.europa.eu/legal-content/EN/TXT...,ProtectedSeas,0.0,None,3,3,3,3,3,1,...,North-West Porcupine Bank Area II Closed Areas,0.0,For updates please see the Protected Sites in ...,3,Year-round,"{'type': 'MultiPolygon', 'coordinates': [[[[-1...",None,1,3,Regulation (EU) 2019/1241 of the European Parl...
188,Regulations Text|http://www.irishstatutebook.i...,National Park and Wildlife Service,0.0,None,3,2,3,2,3,1,...,Codling Fault Zone SAC,0.0,For updates please see the Protected Sites in ...,2,Year-round,"{'type': 'MultiPolygon', 'coordinates': [[[[-5...",None,3,3,S.I. No. 100 of 2016- EUROPEAN UNION HABITATS ...
189,https://eur-lex.europa.eu/legal-content/EN/TXT...,ProtectedSeas,0.0,None,3,3,3,3,3,1,...,Belgica Mound Province Closed Areas,0.0,For updates please see the Protected Sites in ...,3,Year-round,"{'type': 'MultiPolygon', 'coordinates': [[[[-1...",None,1,3,Regulation (EU) 2019/1241 of the European Parl...


### Export Data in Different Formats

In [31]:
def export_PSdata(PSdata):
    """
    Export the PSdata DataFrame to selected file formats with browser downloads in Jupyter.
    
    Parameters:
    -----------
    PSdata : pandas.DataFrame or geopandas.GeoDataFrame
        The dataframe to be exported
    
    Returns:
    --------
    None
        Generates downloadable files for selected formats
    """
    # Available export formats
    export_formats = {
        'Shapefile': '.shp',
        'GeoJSON': '.geojson',
        'GPKG': '.gpkg',
        'CSV': '.csv',
        'SVG': '.svg'
    }
    
    # Create widgets
    format_label = widgets.Label("Select Export Formats:")
    format_checkboxes = {
        format_name: widgets.Checkbox(description=format_name, value=False) 
        for format_name in export_formats.keys()
    }
    
    # Download button and output
    download_button = widgets.Button(description="Generate Downloads")
    output = widgets.Output()
    
    # Arrange widgets
    format_box = widgets.VBox(list(format_checkboxes.values()))
    widget_layout = widgets.VBox([
        format_label, 
        format_box, 
        download_button, 
        output
    ])
    
    def create_download_link(data, filename):
        """
        Create a downloadable link for browser download.
        
        Parameters:
        -----------
        data : bytes
            File content to be downloaded
        filename : str
            Name of the file to be downloaded
        
        Returns:
        --------
        widgets.HTML
            A download link widget
        """
        b64 = base64.b64encode(data).decode()
        html = f'''
        <a download="{filename}" href="data:application/octet-stream;base64,{b64}" download>
            Download {filename}
        </a>
        '''
        return widgets.HTML(html)
    
    def on_download_click(b):
        with output:
            output.clear_output()
            
            # Determine selected formats
            selected_formats = [
                format_name for format_name, checkbox in format_checkboxes.items() 
                if checkbox.value
            ]
            
            if not selected_formats:
                print("❌ Error: No formats selected!")
                return
            
            # Download links to be displayed
            download_links = []
            
            # Export to selected formats
            try:
                for format_name in selected_formats:
                    # Prepare buffer for file content
                    buffer = io.BytesIO()
                    
                    # Export based on format
                    if format_name == 'Shapefile':
                        # Convert regular DataFrame to GeoDataFrame if needed
                        if not isinstance(PSdata, gpd.GeoDataFrame):
                            try:
                                # Attempt to convert to GeoDataFrame 
                                # Assumes you might have geometry column
                                gdf = gpd.GeoDataFrame(PSdata)
                            except Exception:
                                # Fallback: create a simple GeoDataFrame
                                gdf = gpd.GeoDataFrame(
                                    PSdata, 
                                    geometry=gpd.points_from_xy(
                                        PSdata.get('longitude', 0), 
                                        PSdata.get('latitude', 0)
                                    )
                                )
                        else:
                            gdf = PSdata
                        
                        # For Shapefile, we need to save multiple files
                        temp_dir = 'temp_shapefile'
                        os.makedirs(temp_dir, exist_ok=True)
                        shapefile_path = os.path.join(temp_dir, 'PSdata.shp')
                        gdf.to_file(shapefile_path)
                        
                        # Create a zip file of the shapefile components
                        import shutil
                        shutil.make_archive('PSdata_shapefile', 'zip', temp_dir)
                        
                        with open('PSdata_shapefile.zip', 'rb') as f:
                            buffer.write(f.read())
                        
                        # Clean up temporary files
                        shutil.rmtree(temp_dir)
                        os.remove('PSdata_shapefile.zip')
                        
                        download_link = create_download_link(
                            buffer.getvalue(), 
                            f'PSdata{export_formats[format_name]}.zip'
                        )
                    
                    elif format_name in ['GeoJSON', 'GPKG']:
                        # Convert regular DataFrame to GeoDataFrame if needed
                        if not isinstance(PSdata, gpd.GeoDataFrame):
                            try:
                                # Attempt to convert to GeoDataFrame 
                                # Assumes you might have geometry column
                                gdf = gpd.GeoDataFrame(PSdata)
                            except Exception:
                                # Fallback: create a simple GeoDataFrame
                                gdf = gpd.GeoDataFrame(
                                    PSdata, 
                                    geometry=gpd.points_from_xy(
                                        PSdata.get('longitude', 0), 
                                        PSdata.get('latitude', 0)
                                    )
                                )
                        else:
                            gdf = PSdata
                        
                        # Export to specific format
                        driver = 'GeoJSON' if format_name == 'GeoJSON' else 'GPKG'
                        gdf.to_file(buffer, driver=driver)
                        download_link = create_download_link(
                            buffer.getvalue(), 
                            f'PSdata{export_formats[format_name]}'
                        )
                    
                    elif format_name == 'CSV':
                        PSdata.to_csv(buffer, index=False)
                        download_link = create_download_link(
                            buffer.getvalue(), 
                            f'PSdata{export_formats[format_name]}'
                        )
                    
                    elif format_name == 'SVG':
                        import matplotlib.pyplot as plt
                        fig, ax = plt.subplots(figsize=(10, 10))
                        PSdata.plot(ax=ax)
                        plt.savefig(buffer, format='svg')
                        plt.close()
                        download_link = create_download_link(
                            buffer.getvalue(), 
                            f'PSdata{export_formats[format_name]}'
                        )
                    
                    download_links.append(download_link)
                
                # Display download links
                if download_links:
                    print("✅ Downloads generated!")
                    for link in download_links:
                        display(link)
                else:
                    print("❌ No valid downloads could be created.")
            
            except Exception as e:
                print(f"❌ Export Error: {e}")
    
    # Attach click event
    download_button.on_click(on_download_click)
    
    # Display widgets
    display(widget_layout)

In [32]:
warnings.filterwarnings("ignore")

export_PSdata(PSdata)